# Fish Detection and Tracking

In [1]:
# Load library for manipulating data tables. 
import pandas as pd
from datetime import datetime


RESIDENCE_THRESHOLD = 20 # Minutes.

def minute_diff(first_time, last_time):
    first = datetime.strptime(first_time, '%Y-%m-%d %H:%M')
    last = datetime.strptime(last_time, '%Y-%m-%d %H:%M')
    diff = last - first
    return int(diff.total_seconds() / 60)

In [18]:
# Load data in an organized data frame.
# df = pd.read_csv('../../detectionsBCIT_fixed.csv', nrows=1000)
df = pd.read_csv('../../detectionsBCIT_fixed.csv')
# Print the head of the data. 
df.head()

,date_utc,date_local,receiver,acoustic_tag_id,station,gate,rkm,lat,lon,utm_zone,utm_easting,utm_northing
0,2021-09-24 20:46,2021-09-24 13:46,VR2Tx-486856,A69-1602-64231,Swaneset,Lower River Swaneset,11.5,49.29495,-122.657417,10,524909.09,5460301.27
1,2021-09-24 22:05,2021-09-24 15:05,VR2Tx-486856,A69-1602-64231,Swaneset,Lower River Swaneset,11.5,49.29495,-122.657417,10,524909.09,5460301.27
2,2021-09-24 22:15,2021-09-24 15:15,VR2Tx-486856,A69-1602-64231,Swaneset,Lower River Swaneset,11.5,49.29495,-122.657417,10,524909.09,5460301.27
3,2021-09-24 22:41,2021-09-24 15:41,VR2Tx-486856,A69-1602-64231,Swaneset,Lower River Swaneset,11.5,49.29495,-122.657417,10,524909.09,5460301.27
4,2021-09-24 22:55,2021-09-24 15:55,VR2Tx-486856,A69-1602-64231,Swaneset,Lower River Swaneset,11.5,49.29495,-122.657417,10,524909.09,5460301.27


In [19]:
# Strip out unneeded colums. 
dfs = df[['date_local','acoustic_tag_id','station']].copy()
# Print the head of the stripped data frame.
dfs.head()

,date_local,acoustic_tag_id,station
0,2021-09-24 13:46,A69-1602-64231,Swaneset
1,2021-09-24 15:05,A69-1602-64231,Swaneset
2,2021-09-24 15:15,A69-1602-64231,Swaneset
3,2021-09-24 15:41,A69-1602-64231,Swaneset
4,2021-09-24 15:55,A69-1602-64231,Swaneset


In [20]:
# Get list of unique fish
fish = dfs.acoustic_tag_id.unique()
fish

array(['A69-1602-64231', 'A69-1602-64232', 'A69-1602-64234',
       'A69-1602-64235', 'A69-1602-64236', 'A69-1602-64237',
       'A69-1602-64238', 'A69-1602-64241', 'A69-1602-64244',
       'A69-1602-64245', 'A69-1602-64246', 'A69-1602-64247',
       'A69-1602-64248', 'A69-1602-64249', 'A69-1602-64250',
       'A69-1602-64252', 'A69-1602-64255', 'A69-1602-64256',
       'A69-1602-64257', 'A69-1604-24858', 'A69-1604-27419',
       'A69-1604-27420', 'A69-1604-27421', 'A69-1604-27422',
       'A69-1604-27423', 'A69-1604-27424', 'A69-1604-27425',
       'A69-1604-27427', 'A69-1604-27430', 'A69-1604-27431',
       'A69-1604-27432', 'A69-1604-27433', 'A69-1604-27434',
       'A69-1604-27435', 'A69-1604-27436', 'A69-1604-27437',
       'A69-1604-27438', 'A69-1604-27439', 'A69-1604-27440',
       'A69-1604-27441', 'A69-1604-27442', 'A69-1604-27443',
       'A69-1604-27444', 'A69-1604-27445', 'A69-1604-27446',
       'A69-1604-27447', 'A69-1604-27448', 'A69-9001-16062',
       'A69-9001-20978',

## Sort each fish by date. 

In [21]:
for f in fish: 
    print(f)

A69-1602-64231
A69-1602-64232
A69-1602-64234
A69-1602-64235
A69-1602-64236
A69-1602-64237
A69-1602-64238
A69-1602-64241
A69-1602-64244
A69-1602-64245
A69-1602-64246
A69-1602-64247
A69-1602-64248
A69-1602-64249
A69-1602-64250
A69-1602-64252
A69-1602-64255
A69-1602-64256
A69-1602-64257
A69-1604-24858
A69-1604-27419
A69-1604-27420
A69-1604-27421
A69-1604-27422
A69-1604-27423
A69-1604-27424
A69-1604-27425
A69-1604-27427
A69-1604-27430
A69-1604-27431
A69-1604-27432
A69-1604-27433
A69-1604-27434
A69-1604-27435
A69-1604-27436
A69-1604-27437
A69-1604-27438
A69-1604-27439
A69-1604-27440
A69-1604-27441
A69-1604-27442
A69-1604-27443
A69-1604-27444
A69-1604-27445
A69-1604-27446
A69-1604-27447
A69-1604-27448
A69-9001-16062
A69-9001-20978
A69-9001-20980
A69-9001-20981
A69-9001-22495
A69-9001-22498
A69-9001-25438


## For each fish, search for arrivals 

In [22]:
# Print head of stripped data frame.
dfs.head()

,date_local,acoustic_tag_id,station
0,2021-09-24 13:46,A69-1602-64231,Swaneset
1,2021-09-24 15:05,A69-1602-64231,Swaneset
2,2021-09-24 15:15,A69-1602-64231,Swaneset
3,2021-09-24 15:41,A69-1602-64231,Swaneset
4,2021-09-24 15:55,A69-1602-64231,Swaneset


In [23]:
# Count the number of fish
rows_count = len(fish)
print("Number of fish: ", rows_count)

Number of fish:  54


In [24]:
# Filter out current fish
single_fish = dfs.query("acoustic_tag_id == 'A69-1602-64231'")

# List single fish list
single_fish

,date_local,acoustic_tag_id,station
0,2021-09-24 13:46,A69-1602-64231,Swaneset
1,2021-09-24 15:05,A69-1602-64231,Swaneset
2,2021-09-24 15:15,A69-1602-64231,Swaneset
3,2021-09-24 15:41,A69-1602-64231,Swaneset
4,2021-09-24 15:55,A69-1602-64231,Swaneset
...,...,...,...
10886,2023-02-15 17:32,A69-1602-64231,Swaneset
10887,2023-02-15 18:05,A69-1602-64231,Swaneset
10888,2023-02-15 19:40,A69-1602-64231,Swaneset
10889,2023-02-16 1:27,A69-1602-64231,Swaneset


In [25]:
# Print all visited stations for individual fish. 
# stations = single_fish.station.unique()
# stations

output_df = pd.DataFrame(columns=['acoustic_tag_id', 'station', 'date', 'duration'])

In [ ]:
for new_fish in dfs['acoustic_tag_id'].unique():
    filtered_df = dfs[dfs['acoustic_tag_id'] == new_fish]
    sorted_new_fish = filtered_df.sort_values('date_local')
    # print(f"Entries in dataframe with '{new_fish}' in column 'acoustic_tag_id':\n{sorted_new_fish}\n")
    
    print(f"\nFish: {new_fish}")
    
    last_location = "" # Set null location so that first event will start a residence.
    for index, row in sorted_new_fish.iterrows(): # Iterate through all records for single fish. 
        # print(f"Row {index}: {row['date_local']} {row['acoustic_tag_id']} {row['station']}")
#         print(row['station'], row['date_local'])
        if last_location == "": # This is the first record for this fish. 
            arrival_time = row['date_local']
            last_detection = arrival_time
            last_location = row['station']
            first_station = "True"
#             print('no last location')
        else:  # This is not the first record for this fish. 
#             print('In not the first record for this fish')
            time_since_last_detection = minute_diff(last_detection, row['date_local'])
#             print(f"time since last detection: {time_since_last_detection}")
            if last_location != row['station'] or time_since_last_detection > RESIDENCE_THRESHOLD:
                # assertion: residence has ended - calculate duration and write to output dataframe.
#                 print('At the end of residence')
#                 print(f'First station: {first_station}')
                if first_station != "True":
                    date_object = datetime.strptime(last_detection, '%Y-%m-%d %H:%M')
                    date = date_object.date()

                    duration = minute_diff(arrival_time, last_detection)
                    if duration < 0:
                        print(f"Fish: {new_fish}")
                        print(f"Index: {index}")
                        print(f"Arrival time: {arrival_time}")
                        print(f"Last detection: {last_detection}")
                        print(f"Duration: {duration}\n")
                    new_row = [new_fish, last_location, date, duration]
                    output_df.loc[len(output_df)] = new_row
#                     print('Just wrote output_df')
                arrival_time = row['date_local']
                last_location = row['station']
                last_detection = row['date_local']
                first_station = 'False'
            else: # Residence has not ended. 
#                 print('Residence has not ended')
                last_detection = row['date_local'] # Need to save this so we can check if fish is taking too long to return.

output_df.head()


Fish: A69-1602-64231
Fish: A69-1602-64231
Index: 217
Arrival time: 2021-09-25 18:16
Last detection: 2021-09-25 1:59
Duration: -977

Fish: A69-1602-64231
Index: 104
Arrival time: 2021-09-25 23:51
Last detection: 2021-09-25 2:12
Duration: -1299

Fish: A69-1602-64231
Index: 423
Arrival time: 2021-09-26 17:11
Last detection: 2021-09-26 1:59
Duration: -912

Fish: A69-1602-64231
Index: 308
Arrival time: 2021-09-26 21:18
Last detection: 2021-09-26 3:31
Duration: -1067

Fish: A69-1602-64231
Index: 902
Arrival time: 2021-09-27 11:24
Last detection: 2021-09-27 1:56
Duration: -568

Fish: A69-1602-64231
Index: 996
Arrival time: 2021-09-27 20:00
Last detection: 2021-09-27 9:59
Duration: -601

Fish: A69-1602-64231
Index: 1179
Arrival time: 2021-09-28 16:51
Last detection: 2021-09-28 1:56
Duration: -895

Fish: A69-1602-64231
Index: 1072
Arrival time: 2021-09-28 20:01
Last detection: 2021-09-28 4:30
Duration: -931

Fish: A69-1602-64231
Index: 1297
Arrival time: 2021-09-29 10:15
Last detection: 2021-0

Fish: A69-1602-64232
Index: 13504
Arrival time: 2021-10-12 23:59
Last detection: 2021-10-12 4:35
Duration: -1164

Fish: A69-1602-64232
Index: 14547
Arrival time: 2021-10-13 19:48
Last detection: 2021-10-13 1:18
Duration: -1110

Fish: A69-1602-64232
Index: 14558
Arrival time: 2021-10-13 20:08
Last detection: 2021-10-13 2:03
Duration: -1085

Fish: A69-1602-64232
Index: 15332
Arrival time: 2021-10-15 13:36
Last detection: 2021-10-15 1:02
Duration: -754

Fish: A69-1602-64232
Index: 15717
Arrival time: 2021-10-17 19:44
Last detection: 2021-10-17 1:02
Duration: -1122

Fish: A69-1602-64232
Index: 16541
Arrival time: 2021-10-18 19:51
Last detection: 2021-10-18 1:04
Duration: -1127

Fish: A69-1602-64232
Index: 16546
Arrival time: 2021-10-18 22:25
Last detection: 2021-10-18 2:56
Duration: -1169

Fish: A69-1602-64232
Index: 17884
Arrival time: 2021-10-19 17:34
Last detection: 2021-10-19 1:07
Duration: -987

Fish: A69-1602-64232
Index: 17949
Arrival time: 2021-10-19 23:51
Last detection: 2021-10-1

Fish: A69-1602-64232
Index: 38173
Arrival time: 2022-09-13 16:50
Last detection: 2022-09-13 1:16
Duration: -934

Fish: A69-1602-64232
Index: 38182
Arrival time: 2022-09-13 20:01
Last detection: 2022-09-13 2:44
Duration: -1037

Fish: A69-1602-64232
Index: 38236
Arrival time: 2022-09-14 14:41
Last detection: 2022-09-14 1:12
Duration: -809

Fish: A69-1602-64232
Index: 38227
Arrival time: 2022-09-14 21:02
Last detection: 2022-09-14 2:49
Duration: -1093

Fish: A69-1602-64232
Index: 38244
Arrival time: 2022-09-15 14:27
Last detection: 2022-09-15 1:08
Duration: -799

Fish: A69-1602-64232
Index: 38277
Arrival time: 2022-09-16 21:27
Last detection: 2022-09-16 2:55
Duration: -1112

Fish: A69-1602-64232
Index: 38401
Arrival time: 2022-09-17 19:35
Last detection: 2022-09-17 1:57
Duration: -1058

Fish: A69-1602-64232
Index: 38360
Arrival time: 2022-09-17 21:04
Last detection: 2022-09-17 3:30
Duration: -1054

Fish: A69-1602-64232
Index: 38412
Arrival time: 2022-09-18 19:32
Last detection: 2022-09-18

Fish: A69-1602-64232
Index: 41166
Arrival time: 2022-11-09 23:29
Last detection: 2022-11-09 4:30
Duration: -1139

Fish: A69-1602-64232
Index: 41214
Arrival time: 2022-11-10 20:25
Last detection: 2022-11-10 2:05
Duration: -1100

Fish: A69-1602-64232
Index: 41235
Arrival time: 2022-11-12 19:10
Last detection: 2022-11-12 7:40
Duration: -690

Fish: A69-1602-64232
Index: 41234
Arrival time: 2022-11-13 22:29
Last detection: 2022-11-13 7:27
Duration: -902

Fish: A69-1602-64232
Index: 41251
Arrival time: 2022-11-14 18:00
Last detection: 2022-11-14 8:47
Duration: -553

Fish: A69-1602-64232
Index: 41263
Arrival time: 2022-11-17 18:19
Last detection: 2022-11-17 2:45
Duration: -934

Fish: A69-1602-64232
Index: 41265
Arrival time: 2022-11-18 18:47
Last detection: 2022-11-18 1:18
Duration: -1049

Fish: A69-1602-64232
Index: 41289
Arrival time: 2022-11-18 22:41
Last detection: 2022-11-18 4:51
Duration: -1070

Fish: A69-1602-64232
Index: 41304
Arrival time: 2022-11-19 19:23
Last detection: 2022-11-19 

Fish: A69-1602-64234
Index: 44542
Arrival time: 2021-10-31 11:59
Last detection: 2021-10-31 1:36
Duration: -623

Fish: A69-1602-64234
Index: 44605
Arrival time: 2021-10-31 21:34
Last detection: 2021-10-31 6:46
Duration: -888

Fish: A69-1602-64234
Index: 44655
Arrival time: 2021-11-01 13:58
Last detection: 2021-11-01 2:53
Duration: -665

Fish: A69-1602-64234
Index: 44690
Arrival time: 2021-11-02 13:24
Last detection: 2021-11-02 2:41
Duration: -643

Fish: A69-1602-64234
Index: 44710
Arrival time: 2021-11-03 15:23
Last detection: 2021-11-03 3:09
Duration: -734

Fish: A69-1602-64234
Index: 44738
Arrival time: 2021-11-04 20:12
Last detection: 2021-11-04 6:32
Duration: -820

Fish: A69-1602-64234
Index: 44772
Arrival time: 2021-11-05 21:25
Last detection: 2021-11-05 6:19
Duration: -906

Fish: A69-1602-64234
Index: 44767
Arrival time: 2021-11-06 21:48
Last detection: 2021-11-06 5:54
Duration: -954

Fish: A69-1602-64234
Index: 44834
Arrival time: 2021-11-07 20:55
Last detection: 2021-11-07 6:36

...

Fish: A69-1602-64236
Index: 100843
Arrival time: 2022-09-27 19:16
Last detection: 2022-09-27 1:59
Duration: -1037

Fish: A69-1602-64236
Index: 100822
Arrival time: 2022-09-27 23:45
Last detection: 2022-09-27 2:30
Duration: -1275

Fish: A69-1602-64236
Index: 100885
Arrival time: 2022-09-28 23:34
Last detection: 2022-09-28 7:16
Duration: -978

Fish: A69-1602-64236
Index: 100949
Arrival time: 2022-09-29 19:16
Last detection: 2022-09-29 1:27
Duration: -1069

Fish: A69-1602-64236
Index: 100943
Arrival time: 2022-09-29 22:37
Last detection: 2022-09-29 3:33
Duration: -1144

Fish: A69-1602-64236
Index: 101062
Arrival time: 2022-09-30 17:03
Last detection: 2022-09-30 1:32
Duration: -931

Fish: A69-1602-64236
Index: 101291
Arrival time: 2022-10-01 22:03
Last detection: 2022-10-01 4:23
Duration: -1060

Fish: A69-1602-64236
Index: 101472
Arrival time: 2022-10-02 17:32
Last detection: 2022-10-02 1:03
Duration: -989

Fish: A69-1602-64236
Index: 101388
Arrival time: 2022-10-02 20:00
Last detection: 2

Fish: A69-1602-64237
Index: 108377
Arrival time: 2022-05-20 23:07
Last detection: 2022-05-20 2:06
Duration: -1261

Fish: A69-1602-64237
Index: 108395
Arrival time: 2022-05-22 23:07
Last detection: 2022-05-22 5:57
Duration: -1030

Fish: A69-1602-64237
Index: 108404
Arrival time: 2022-05-23 22:28
Last detection: 2022-05-23 2:35
Duration: -1193

Fish: A69-1602-64237
Index: 108409
Arrival time: 2022-05-24 23:48
Last detection: 2022-05-24 2:50
Duration: -1258

Fish: A69-1602-64237
Index: 108427
Arrival time: 2022-05-25 17:56
Last detection: 2022-05-25 3:44
Duration: -852

Fish: A69-1602-64237
Index: 108442
Arrival time: 2022-05-26 19:39
Last detection: 2022-05-26 1:06
Duration: -1113

Fish: A69-1602-64237
Index: 108459
Arrival time: 2022-05-27 23:25
Last detection: 2022-05-27 4:52
Duration: -1113

Fish: A69-1602-64237
Index: 108469
Arrival time: 2022-05-28 19:35
Last detection: 2022-05-28 2:25
Duration: -1030

Fish: A69-1602-64237
Index: 108488
Arrival time: 2022-05-30 15:07
Last detection:

Fish: A69-1602-64244
Index: 112703
Arrival time: 2023-01-30 14:38
Last detection: 2023-01-30 7:33
Duration: -425

Fish: A69-1602-64244
Index: 112915
Arrival time: 2023-02-10 22:23
Last detection: 2023-02-10 6:57
Duration: -926

Fish: A69-1602-64244
Index: 113038
Arrival time: 2023-02-11 22:53
Last detection: 2023-02-11 9:50
Duration: -783

Fish: A69-1602-64244
Index: 113098
Arrival time: 2023-02-12 12:40
Last detection: 2023-02-12 2:43
Duration: -597

Fish: A69-1602-64244
Index: 113080
Arrival time: 2023-02-13 14:07
Last detection: 2023-02-13 1:06
Duration: -781


Fish: A69-1602-64245
Fish: A69-1602-64245
Index: 113119
Arrival time: 2022-04-11 10:05
Last detection: 2022-04-11 9:57
Duration: -8

Fish: A69-1602-64245
Index: 113930
Arrival time: 2022-10-18 10:01
Last detection: 2022-10-18 1:55
Duration: -486

Fish: A69-1602-64245
Index: 113430
Arrival time: 2022-10-18 23:02
Last detection: 2022-10-18 4:45
Duration: -1097

Fish: A69-1602-64245
Index: 114117
Arrival time: 2022-10-19 17:45
L

Fish: A69-1602-64245
Index: 123947
Arrival time: 2023-01-11 19:33
Last detection: 2023-01-11 1:58
Duration: -1055

Fish: A69-1602-64245
Index: 123973
Arrival time: 2023-01-11 20:00
Last detection: 2023-01-11 9:58
Duration: -602

Fish: A69-1602-64245
Index: 124259
Arrival time: 2023-01-13 15:38
Last detection: 2023-01-13 9:49
Duration: -349

Fish: A69-1602-64245
Index: 124777
Arrival time: 2023-01-14 10:01
Last detection: 2023-01-14 1:59
Duration: -482

Fish: A69-1602-64245
Index: 124893
Arrival time: 2023-01-14 20:00
Last detection: 2023-01-14 9:59
Duration: -601

Fish: A69-1602-64245
Index: 125144
Arrival time: 2023-01-15 18:53
Last detection: 2023-01-15 1:55
Duration: -1018

Fish: A69-1602-64245
Index: 125259
Arrival time: 2023-01-15 20:42
Last detection: 2023-01-15 8:58
Duration: -704

Fish: A69-1602-64245
Index: 125461
Arrival time: 2023-01-16 10:21
Last detection: 2023-01-16 1:50
Duration: -511

Fish: A69-1602-64245
Index: 125506
Arrival time: 2023-01-16 22:17
Last detection: 2023

Fish: A69-1602-64247
Index: 144667
Arrival time: 2022-11-27 17:46
Last detection: 2022-11-27 1:58
Duration: -948

Fish: A69-1602-64247
Index: 144777
Arrival time: 2022-11-27 20:00
Last detection: 2022-11-27 9:58
Duration: -602

Fish: A69-1602-64247
Index: 145132
Arrival time: 2022-11-28 15:58
Last detection: 2022-11-28 1:58
Duration: -840

Fish: A69-1602-64247
Index: 145358
Arrival time: 2022-11-28 23:12
Last detection: 2022-11-28 9:09
Duration: -843

Fish: A69-1602-64247
Index: 145209
Arrival time: 2022-11-29 21:09
Last detection: 2022-11-29 3:58
Duration: -1031

Fish: A69-1602-64247
Index: 146133
Arrival time: 2022-11-30 10:00
Last detection: 2022-11-30 1:58
Duration: -482

Fish: A69-1602-64247
Index: 146236
Arrival time: 2022-11-30 20:00
Last detection: 2022-11-30 9:57
Duration: -603

Fish: A69-1602-64247
Index: 146730
Arrival time: 2022-12-01 16:38
Last detection: 2022-12-01 1:57
Duration: -881

Fish: A69-1602-64247
Index: 146801
Arrival time: 2022-12-01 21:05
Last detection: 2022-

## Save dataframe. 

In [28]:
output_df.to_csv("../../WSP_Residence_Time.csv")